# **Scraping**
## imports

In [ ]:
!apt update
!pip install selenium
!pip install openpyxl
!pip install xlsxwriter
!pip install chromium-chromedriver
!pip install pytz

from datetime import datetime
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

from datetime import datetime, timedelta
import pytz
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)

## Extraccion de datos
- equipos -> [[equipos_locales], [equipos_visitantes]]
- cuotas -> [[cuota_local, cuota_empate, cuota_visitante]]
- fecha_hora -> [fecha, hora]
- dobles oportunidades -> [[Lo/Em, Lo/Vi, Vi/Em]]
- ambos_anotan -> [[si, no]]

In [ ]:
class Scraping:
    def __init__(self, url) -> None:
        print(f"analizando >> {url} <<")
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'lxml')
        self.container = soup.find('div', class_='fragment expander coupon-for-type')

        duplicade_links = [elem.get("href") for elem in self.container.find_all('a', title="Número de mercados")]
        self.links_per_match = []

        for item in duplicade_links:
            if item not in self.links_per_match:
                self.links_per_match.append(item)

        #variable para la liga
        self.liga = url.split('/')[-1]
    def equipos(self):
        local_vist = []
        for link in self.links_per_match:
            new_url = "https://apuestas.wplay.co" + link
            driver.get(new_url)
            time.sleep(6)
            names = driver.find_elements(By.CLASS_NAME, "sr-lmt-plus-scb__team-name")
            local_vist.append([names[0].text, names[1].text])
        return local_vist


    def cuotas(self):
        puntos = [elem.text for elem in self.container.find_all('span', class_='price dec')]
        cuotas = [puntos[i:i+3] for i in range(0, len(puntos), 3)]
        for i in range(len(cuotas)):
            for j in range(len(cuotas[i])):
                val = cuotas[i][j]
                val = val.replace('.',',')
                cuotas[i][j] = val
        return cuotas

    def hora_fecha(self):
        meses = {'Ene': '01', 'Feb': '02', 'Mar': '03', 'Abr': '04', 'May': '05', 'Jun': '06',
             'Jul': '07', 'Ago': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dic': '12'}
        horarios = []
        eventos = self.container.find_all('div', {'class': 'ev'})
        for evento in eventos:
            hora_element = evento.find('span', {'class': 'time'})
            fecha_element = evento.find('span', {'class': 'date'})

            if hora_element and fecha_element:
                hora = hora_element.text
                fecha = fecha_element.text
                fecha = fecha + ' 2023'
                fecha = fecha.replace(fecha.split()[1], meses[fecha.split()[1]])
                fecha_hora = fecha + ' ' + hora
                fecha_hora = datetime.strptime(fecha_hora, '%d %m %Y %H:%M')
                fecha_hora = datetime.strftime(fecha_hora, '%Y-%m-%d %H:%M')
                horarios.append(str(fecha_hora).split(' '))
        return horarios

    def dobles_oportunidades(self):
        dobles_oportunidades = []
        for link in self.links_per_match:
            try:
                new_url = "https://apuestas.wplay.co" + link
                new_page = requests.get(new_url)
                new_soup = BeautifulSoup(new_page.content, 'lxml')
                new_container = new_soup.find('ul', class_='default mkt_content limited mkt-sort-DBLC ev-sort-MT')
                group = [elem.text for elem in new_container.find_all('span', class_="price dec")]
                #remplazando . por ,
                for i in range(len(group)):
                    val = group[i].replace('.',',')
                    group[i] = val
                dobles_oportunidades.append(group)
            except AttributeError:
                print(f".  el link: -- {new_url} -- no contiene dobles_oportunidades")
                dobles_oportunidades.append(["N/A","N/A", "N/A"])
        return dobles_oportunidades

    def ambos_anotan(self):
        si_no = []
        for link in self.links_per_match:
            try:
                new_url = "https://apuestas.wplay.co" + link
                new_page = requests.get(new_url)
                new_soup = BeautifulSoup(new_page.content, 'lxml')
                new_container = new_soup.find('table', class_='horizontal mkt_content mkt-sort-BTSC')
                group = [elem.text for elem in new_container.find_all('span', class_="price dec")]
                for i in range(len(group)):
                    val = group[i].replace('.',',')
                    group[i] = val
                si_no.append(group)
            except AttributeError:
                print(f".  el link: -- {new_url} -- no contiene ambos_anotan")
                si_no.append(["N/A","N/A"])
        return si_no

## Datos pre-patido

In [ ]:
class Excel_pre_partido():
    def __init__(self, urls) -> None:
        ligas_totales = []
        for url in urls:
            try:
                obj = Scraping(url)
                equipos = obj.equipos()
                cuotas = obj.cuotas()
                hora_fecha = obj.hora_fecha()
                dobles_oportunidades = obj.dobles_oportunidades()
                si_no = obj.ambos_anotan()
                self.liga = obj.liga

                total_data = zip(equipos, cuotas, hora_fecha,
                                dobles_oportunidades, si_no)
                ligas_totales.extend(self.data_per_liga(total_data))
            except AttributeError:
                print("!NO tiene partidos¡")

            COLUMNS = ['liga', 'local', 'visitante', 'cuo_local', 'cuo_emp', 'cuo_vist', 'fecha', 'hora',
                    'local_emp', 'local_vist', 'vist_emp', 'si_anotan', 'no_anotan']

            self.Excel(ligas_totales, COLUMNS)


    def data_per_liga(self, total_data):
        partidos = []
        for i in total_data:
            local, vist = i[0]
            cuo_local, cuo_emp, cuo_vist = i[1]
            fecha, hora = i[2]
            local_emp, local_vist, vist_emp = i[3]
            si_anotan, no_anotan = i[4]

            datos_part = (self.liga, local, vist, cuo_local, cuo_emp, cuo_vist, fecha, hora,
                          local_emp, local_vist, vist_emp, si_anotan, no_anotan)
            partidos.append(datos_part)
        return partidos

    def Excel(self, data, columns):
        df =pd.DataFrame(data, columns=columns)

        nuevos_nombre = {
            'vist':'visitante',
            'cuo_local':'cuota_local',
            'cuo_emp':'cuota_empate',
            'cuo_vist':'cuota_visitante',
            'local_emp':'local/empate',
            'local_vist':'local/visitante',
            'vist_emp':'visitante/empate'
        }
        df = df.rename(columns=nuevos_nombre)

        columnas_ordenadas = ['liga', 'fecha', 'hora', 'local','cuota_local', 'vs',
                              'cuota_empate','visitante','cuota_visitante','si_anotan', 'no_anotan',
                              'local/empate', 'local/visitante', 'visitante/empate']
        df = df.reindex(columns=columnas_ordenadas)

        writer = pd.ExcelWriter('cuotas_pre_partido.xlsx', engine='xlsxwriter')
        df.to_excel(writer, sheet_name='Cuotas_1', index=False)
        writer.close()

## Datos pos-partido

In [ ]:
from datetime import datetime, timedelta
import re

class Excel_pos_partido():
    def __init__(self, excel, urls) -> None:
        try:
            df = pd.read_excel(excel, engine='openpyxl')

            columnas_interes = ['liga', 'hora', 'local', 'visitante', 'fecha']
            df_interes = df[columnas_interes]
            lista_general = []

            grupos_por_liga = df_interes.groupby('liga', sort=False)

            for liga, grupo in grupos_por_liga:
                lista_liga = grupo.values.tolist()
                lista_general.append(lista_liga)

            assert len(lista_general) == len(urls)

            columna_goles = []
            for i in range(len(urls)):
                columna_goles.extend(self.get_goles(urls[i], lista_general[i]))
            self.made_excel(df, columna_goles)

        except FileNotFoundError:
            print(f"El archivo '{excel}' no fue encontrado.")
        except AssertionError:
            print(f"se tienen {len(lista_general)} ligas, y {len(urls)} urls \n ingrese el mismo numero de ligas que de Urls")
        except Exception as e:
            print(f"Ocurrió un error al abrir el archivo de Excel: {e}")

    def get_goles(self, url, liga):
        columna_goles = []
        fechas = self.get_link(url, liga[1][4])
        print(f"analizanzo: >>{liga[1][0]}<<")

        driver = webdriver.Chrome(options=options)
        driver.get(fechas)
        time.sleep(10)
        raw_rows = driver.find_elements(By.CLASS_NAME, 'cursor-pointer')

        rows = []
        for i in raw_rows:
            if i.find_elements(By.CLASS_NAME, 'mobile-width-5.text-center'):
                rows.append(i)

        for i in range(len(liga)):
            TEMPORAL = 0
            _ = 0
            for row in rows:
                row_hora, row_local, row_visit = self.datos_per_row(row)

                #hora_liga.strip() == row_hora.strip() and
                if row_local.strip() == liga[i][2].strip() and row_visit.strip() == liga[i][3].strip():
                    try:
                        _ += 1
                        goles = row.find_elements(By.CLASS_NAME, "hidden-xs-up.visible-sm-up.no-wrap")[1]
                        goles = goles.get_attribute('outerHTML')
                        soup = BeautifulSoup(goles, 'html.parser')
                        goles = soup.get_text()
                        value_row = self.evaluar_goles(puntuador=goles)
                        if _ < 2:
                          columna_goles.append(value_row)
                        else:
                          columna_goles[-1] = value_row
                        continue

                    except Exception as e:
                        value_row = self.evaluar_goles(puntuador=None)
                        if _ < 2:
                          columna_goles.append(value_row)
                        else:
                          columna_goles[-1] = value_row
                        continue
                else:
                    TEMPORAL += 1
                    if TEMPORAL >= len(rows):
                        value_row = self.evaluar_goles(puntuador=None)
                        columna_goles.append(value_row)
        return columna_goles

#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    def evaluar_goles(self, puntuador):
        if puntuador == None:
            default = ['RES', "E", "AM", "N/A", "X", "N/A"]
            return default
        else:
            goles = puntuador.split(':')
            uno = int(goles[0])
            dos = int(goles[1])
            RES = lambda uno, dos: "EM" if uno == dos else ("GL" if uno > dos else "GV")
            E = lambda uno, dos: "SI" if uno == dos else "NO"
            AM = lambda uno, dos : "NO" if uno == 0 or dos == 0 else "SI"
            resultado = [RES(uno,dos), E(uno,dos), AM(uno,dos), uno, "X", dos]
            return resultado

    #obtiene los datos de hora, local y visitante de cada ilera
    def datos_per_row(self, row):
        hora_row = row.find_element(By.CLASS_NAME, 'mobile-width-5.text-center')
        hora_row = hora_row.get_attribute('outerHTML')
        soup = BeautifulSoup(hora_row, 'html.parser')
        hora_row = soup.get_text()

        equipos = []
        espacios = row.find_elements(By.CLASS_NAME, "hidden-xs-up.visible-sm-up.wrap")
        for i in espacios:
            i = i.get_attribute("outerHTML")
            soup = BeautifulSoup(i, 'html.parser')
            equipos.append(soup.get_text())

        return hora_row, equipos[0], equipos[1]

    def get_link(self, url, fecha):
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'lxml')
        statistics = soup.find('a', class_="stats").get('href')

        fecha = fecha.split('-')
        fecha = "-".join(fecha[:2])

        page = requests.get(statistics)
        soup = BeautifulSoup(page.content, 'lxml')
        fechas = soup.find('a', class_='btn btn-top-menu active').get('href').replace('headtohead', 'fixtures')
        fechas = 'https://statistics.wplay.co' + fechas + '/month/' + fecha
        return fechas

    def made_excel(self, df, goles):
        df = df
        nombres_columnas = ['RES', 'E', 'AM', '1', 'X', '2']
        for i, nombre_columna in enumerate(nombres_columnas):
            df[nombre_columna] = [item[i] for item in goles]
        columnas_ordenadas = ['liga', 'fecha', 'hora', 'RES', 'E', 'AM', '1', 'X', '2',
                              'local','cuota_local', 'vs',
                              'cuota_empate','visitante','cuota_visitante','si_anotan', 'no_anotan',
                              'local/empate', 'local/visitante', 'visitante/empate']
        df = df.reindex(columns=columnas_ordenadas)

        writer = pd.ExcelWriter('cuotas_pos_partido.xlsx', engine='xlsxwriter')
        df.to_excel(writer, sheet_name='Cuotas_1', index=False)
        writer.close()

## RESULTADOS

In [ ]:
#---ingrese los links en las listas----
total_urls = ['https://apuestas.wplay.co/es/PremierLeague',
        'https://apuestas.wplay.co/es/t/19160/La-Liga',
        'https://apuestas.wplay.co/es/Ligue1',
        "https://apuestas.wplay.co/es/ItaliaSerieA",
        'https://apuestas.wplay.co/es/Bundesliga',
        'https://apuestas.wplay.co/es/t/19344/Bundesliga-2',
        'https://apuestas.wplay.co/es/PrimeraAColombia',
        'https://apuestas.wplay.co/es/t/19296/Argentina-Copa-de-la-Liga-Profesional',
        'https://apuestas.wplay.co/es/t/19398/Uruguay-Primera-Divisi%C3%B3n',
        'https://apuestas.wplay.co/es/t/19359/Paraguay-Primera-Divisi%C3%B3n',
        'https://apuestas.wplay.co/es/t/19211/Portugal-Primeira-Liga',
        'https://apuestas.wplay.co/es/t/19372/B%C3%A9lgica-1ra-Divisi%C3%B3n-A',
        'https://apuestas.wplay.co/es/t/19358/Holanda-Eredivisie',
        'https://apuestas.wplay.co/es/MLS',
        'https://apuestas.wplay.co/es/t/29812/Colombia-Primera-B',
        'https://apuestas.wplay.co/es/t/19373/Ecuador-Primera-A',
        'https://apuestas.wplay.co/es/t/48352/Segunda',
        'https://apuestas.wplay.co/es/t/19303/Chile-Primera-Divisi%C3%B3n',
        'https://apuestas.wplay.co/es/LigaMX',]

a = ['https://apuestas.wplay.co/es/t/29812/Colombia-Primera-B',
     'https://apuestas.wplay.co/es/PrimeraAColombia',
     'https://apuestas.wplay.co/es/t/19296/Argentina-Copa-de-la-Liga-Profesional',
     'https://apuestas.wplay.co/es/LigaMX',]

#ingrese el nombre del excel a analizar
excel = "cuotas_pre_partido_1.xlsx"


### **Excel pre partido**
**solo recive los URLS**

Esta llamada crea un excel con los partidos de las ligas que ingrese, sacando datos como:
- liga
- fecha, hora, local, visitante
- cuotas
- ambos anotan
- Doble oportunidad

In [ ]:
PRE_PARTIDO = Excel_pre_partido(urls=total_urls)

### **Excel pos partido**
**recive como parametros las URLS y el EXCEL**
>**Importante:** el numero de **urls** debe ser igual al numero de **ligas del excel**

> el excel debe estar disponibe en la pestaña de **archivos**

Esta llamada modifica el excel prepartido añadiendo:
- goles (hallan jugado o no)
- todos los datos del excel pre partdo

In [ ]:
POS_PARTIDO = Excel_pos_partido(urls=a, excel=excel)